In [1]:
import networkx as nx
from utils.graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [2]:
static = pd.read_csv("/home/mikolaj/git/pos-prediction-graph/data_generation/mega_static.csv")
detected = pd.read_csv("/home/mikolaj/git/pos-prediction-graph/data_generation/mega_randomized.csv")

In [3]:
componenets = detected.columns
componenets = [
    x
    for x in componenets
    if x
    not in [
        "name",
        "number",
        "x",
        "y",
        "z",
        "id",
        "parent",
        "distance_to_parent",
    ]
]

In [4]:
componenets

['conn_a',
 'conn_f_30',
 'conn_f_37',
 'conn_f_60',
 'conn_e',
 'conn_c',
 'bag_b_l',
 'box',
 'cross_gh',
 'conn_g',
 'conn_h',
 'conn_k',
 'conn_d',
 'bag_b_r',
 'bag_a_r',
 'conn_b',
 'conn_i',
 'conn_j',
 'bag_a_l']

In [5]:
level_order = [
    "root_cross_right",
    "root_cross_left",
    "root_cross_right2",
    "conn_k",
    "cross_e",
    "cross_bag_br",
    "cross_ij1",
    "cross_e_c",
    "cross_f",
    "bag_b_r",
    "bag_a_r",
    "conn_b",
    "cross_ij2",
    "conn_e",
    "conn_c",
    "cross_k",
    "cross_f30",
    "conn_i",
    "conn_j",
    "box",
    "conn_a",
    "conn_f_30",
    "cross_f37",
    "conn_f_60",
    "bag_b_l",
    "cross_gh",
    "conn_f_37",
    "conn_d",
    "conn_g",
    "conn_h",
    "bag_a_l",
]

In [6]:
quadratic_model = Model("quadratic")

Restricted license - for non-production use only - expires 2024-10-28


In [7]:
variables = {}
for name in level_order:
    variables[f"{name}_x"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_x"
    )
    variables[f"{name}_y"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_y"
    )
    variables[f"{name}_z"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_z"
    )

In [8]:
componenets

['conn_a',
 'conn_f_30',
 'conn_f_37',
 'conn_f_60',
 'conn_e',
 'conn_c',
 'bag_b_l',
 'box',
 'cross_gh',
 'conn_g',
 'conn_h',
 'conn_k',
 'conn_d',
 'bag_b_r',
 'bag_a_r',
 'conn_b',
 'conn_i',
 'conn_j',
 'bag_a_l']

In [9]:
obj_fn = 0
for row in detected.iterrows():
    detected_x = row[1]["x"]
    detected_y = row[1]["y"]
    detected_z = row[1]["z"]
    for component in componenets:
        confidence = row[1][component]
        term = confidence * (
            (variables[f"{component}_x"] - detected_x) ** 2
            + (variables[f"{component}_y"] - detected_y) ** 2
            + (variables[f"{component}_z"] - detected_z) ** 2
        )
        obj_fn += term
    

In [10]:
for name in level_order:
    node_info = static[static["name"] == name]
    parent = node_info["parent"].values[0]
    distance_to_parent = node_info["distance_to_parent"].values[0]
    if name == "root_cross_left":
        quadratic_model.addQConstr(variables[f"{name}_x"] == -13)
        quadratic_model.addQConstr(variables[f"{name}_y"] == -20)
        quadratic_model.addQConstr(variables[f"{name}_z"] == 0)

    elif name == "root_cross_right":
        quadratic_model.addQConstr(variables[f"{name}_x"] == 10)
        quadratic_model.addQConstr(variables[f"{name}_y"] == -1)
        quadratic_model.addQConstr(variables[f"{name}_z"] == 0)
    else:
        quadratic_model.addQConstr(
            (
                (variables[f"{name}_x"] - variables[f"{parent}_x"]) ** 2
                + (variables[f"{name}_y"] - variables[f"{parent}_y"]) ** 2
                + (variables[f"{name}_z"] - variables[f"{parent}_z"]) ** 2
            )
            <= distance_to_parent**2
        )

In [11]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)

In [12]:
quadratic_model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 93 columns and 6 nonzeros
Model fingerprint: 0x7dd62a6d
Model has 57 quadratic objective terms
Model has 29 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [5e+00, 2e+03]
  QObjective range [2e+01, 2e+01]
  Bounds range     [2e+02, 2e+02]
  RHS range        [1e+00, 2e+01]
  QRHS range       [2e+01, 1e+04]
Presolve removed 6 rows and 6 columns
Presolve time: 0.01s
Presolved: 240 rows, 202 columns, 409 nonzeros
Presolved model has 30 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.521e+04
 Factor NZ  : 1.604e+04
 Factor Ops : 1.760e+06 (less than 1 second per iteration)
 Threads    : 1

                

In [13]:
import networkx as nx
from utils.graph_utils import plot_graph
import numpy as np
import pandas as pd


G1 = nx.Graph()
for i in range(len(detected)):
    G1.add_node(
        str(detected["id"][i]), pos=(detected["x"][i], detected["z"][i], detected["y"][i])
    )

# G1.add_node("conn_a", pos=(10, 10, 0))

G2 = nx.Graph()
for name in level_order:
    # if x and y and z are  different than 250 or -250
    if variables[f"{name}_x"].x != 250 and variables[f"{name}_x"].x != -250:
        # print(name, variables[f"{name}_x"].x, variables[f"{name}_y"].x, variables[f"{name}_z"].x)
        G2.add_node(
            name,
            pos=(
                variables[f"{name}_x"].x,
                variables[f"{name}_z"].x,
                variables[f"{name}_y"].x,
            ),
        )


plot_graph(static_graph=G1, detected_graph=G2, name="qcqp2", save=True)

Graph saved to qcqp2.html


In [14]:
def calculate_distance(point1, point2):
    """Calculates the Euclidean distance between two 3D points"""
    x1, y1, z1 = point1
    x2, y2, z2 = point2
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

def find_closest_node(G1, G2):
    """Finds the closest G1 node for each node in G2"""
    closest_nodes = {}
    for g2_node in G2.nodes(data=True):
        g2_pos = g2_node[1]['pos']
        min_dist = float('inf')
        min_node = None
        for g1_node in G1.nodes(data=True):
            g1_pos = g1_node[1]['pos']
            dist = calculate_distance(g1_pos, g2_pos)
            if dist < min_dist:
                min_dist = dist
                min_node = g1_node[0]
        closest_nodes[g2_node[0]] = min_node
    return closest_nodes

# Call the function
closest_nodes = find_closest_node(G2, G1)

# If you want to see the result in a DataFrame
df_closest_nodes = pd.DataFrame(closest_nodes.items(), columns=['G1_Node', 'Closest_G2_Node'])
df_closest_nodes = df_closest_nodes.set_index('G1_Node')



In [15]:
#sort by column "closest_G2_node"
df_closest_nodes.sort_values(by=['Closest_G2_Node'], inplace=True)

In [16]:
detected['valid'] = detected['id'].astype(str).map(df_closest_nodes['Closest_G2_Node'])


In [17]:
final_model = Model("quadratic")

In [18]:
f_vars = {}
for name in level_order:
    f_vars[f"{name}_x"] = final_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_x"
    )
    f_vars[f"{name}_y"] = final_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_y"
    )
    f_vars[f"{name}_z"] = final_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_z"
    )

In [19]:
obj_fn = 0
for row in detected.iterrows():
    detected_x = row[1]["x"]
    detected_y = row[1]["y"]
    detected_z = row[1]["z"]
    component = row[1]["valid"]
    term = (
        (f_vars[f"{component}_x"] - detected_x) ** 2
        + (f_vars[f"{component}_y"] - detected_y) ** 2
        + (f_vars[f"{component}_z"] - detected_z) ** 2
    )
    obj_fn += term
    

In [20]:
for name in level_order:
    node_info = static[static["name"] == name]
    parent = node_info["parent"].values[0]
    distance_to_parent = node_info["distance_to_parent"].values[0]
    if name == "root_cross_left":
        final_model.addQConstr(f_vars[f"{name}_x"] == -13)
        final_model.addQConstr(f_vars[f"{name}_y"] == -20)
        final_model.addQConstr(f_vars[f"{name}_z"] == 0)

    elif name == "root_cross_right":
        final_model.addQConstr(f_vars[f"{name}_x"] == 10)
        final_model.addQConstr(f_vars[f"{name}_y"] == -1)
        final_model.addQConstr(f_vars[f"{name}_z"] == 0)
    else:
        final_model.addQConstr(
            (
                (f_vars[f"{name}_x"] - f_vars[f"{parent}_x"]) ** 2
                + (f_vars[f"{name}_y"] - f_vars[f"{parent}_y"]) ** 2
                + (f_vars[f"{name}_z"] - f_vars[f"{parent}_z"]) ** 2
            )
            <= distance_to_parent**2
        )

In [21]:
final_model.setObjective(obj_fn, GRB.MINIMIZE)
final_model.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 6 rows, 93 columns and 6 nonzeros
Model fingerprint: 0x8b356088
Model has 63 quadratic objective terms
Model has 29 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 2e+00]
  Objective range  [5e+00, 3e+03]
  QObjective range [2e+00, 4e+01]
  Bounds range     [2e+02, 2e+02]
  RHS range        [1e+00, 2e+01]
  QRHS range       [2e+01, 1e+04]
Presolve removed 6 rows and 6 columns
Presolve time: 0.01s
Presolved: 252 rows, 202 columns, 421 nonzeros
Presolved model has 30 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.902e+04
 Factor NZ  : 1.967e+04
 Factor Ops : 2.466e+06 (less than 1 second per iteration)
 Threads    : 1

                

In [22]:
import networkx as nx
from utils.graph_utils import plot_graph
import numpy as np
import pandas as pd


G1 = nx.Graph()
for i in range(len(static)):
    G1.add_node(static["name"][i], pos=(static["x"][i], static["z"][i], static["y"][i]))
for i in range(len(static)):
    if static["parent"][i] != 0 and static["parent"][i] != "0":
        G1.add_edge(static["name"][i], static["parent"][i])


G2 = nx.Graph()
for name in level_order:
    # if x and y and z are  different than 250 or -250
    if f_vars[f"{name}_x"].x != 250 and f_vars[f"{name}_x"].x != -250:
        # print(name, variables[f"{name}_x"].x, variables[f"{name}_y"].x, variables[f"{name}_z"].x)
        G2.add_node(
            name,
            pos=(
                f_vars[f"{name}_x"].x,
                f_vars[f"{name}_z"].x,
                f_vars[f"{name}_y"].x,
            ),
        )


plot_graph(static_graph=G1, detected_graph=G2, name="qcqp3", save=True)

Graph saved to qcqp3.html


In [23]:
G1 = nx.Graph()
for i in range(len(static)):
    G1.add_node(static["name"][i], pos=(static["x"][i], static["z"][i], static["y"][i]))
for i in range(len(static)):
    if static["parent"][i] != 0 and static["parent"][i] != "0":
        G1.add_edge(static["name"][i], static["parent"][i])

G2 = nx.Graph()
for i in range(len(detected)):
    G2.add_node(
        str(detected["id"][i]), pos=(detected["x"][i], detected["z"][i], detected["y"][i])
    )

plot_graph(static_graph=G1, detected_graph=G2, name="qcqp1", save=True)

Graph saved to qcqp1.html
